<a href="https://colab.research.google.com/github/lucandro/IBM/blob/main/Recogida_de_datos_con_Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicción del aterrizaje de la primera etapa del Falcon 9 de Space X
Extracción de datos web de los registros de lanzamientos del Falcon 9 y el Falcon Heavy de Wikipedia

Tiempo necesario estimado: 40 minutos

En este laboratorio, realizarás una extracción de datos web para recopilar registros históricos de lanzamientos del Falcon 9 de una página de Wikipedia titulada Lista de lanzamientos del Falcon 9 y el Falcon Heavy

[Wikipedia](https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches)


## Objetivos

Recopilar registros de lanzamiento de Falcon 9 con BeautifulSoup:

Extraer una tabla HTML de registros de lanzamiento de Falcon 9 de Wikipedia
Analizar la tabla y convertirla en un marco de datos de Pandas

Primero, importemos los paquetes necesarios para este laboratorio.

In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

y le proporcionaremos algunas funciones de ayuda para procesar la tabla HTML extraída de la web.

In [3]:
def date_time(table_cells):
    """
    Esta función devuelve los datos y la hora de la celda de la tabla HTML
    Entrada: el elemento de una celda de datos de la tabla extrae una fila adicional
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    Esta función devuelve la versión de refuerzo de la celda de la tabla HTML
    Entrada: el elemento de una celda de datos de la tabla extrae una fila adicional
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    Esta función devuelve el estado de aterrizaje de la celda de la tabla HTML
    Entrada: el elemento de una celda de datos de la tabla extrae una fila adicional
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    Esta función devuelve el estado de aterrizaje de la celda de la tabla HTML
    Entrada: el elemento de una celda de datos de la tabla extrae una fila adicional
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()

    colunm_name = ' '.join(row.contents)

    # Filtrar el dígito y los nombres vacíos
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name


Para mantener la coherencia de las tareas del laboratorio, se le pedirá que extraiga los datos de una instantánea de la página Wiki de la Lista de lanzamientos de Falcon 9 y Falcon Heavy actualizada el 9 de junio de 2021

In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

A continuación, solicite la página HTML desde la URL anterior y obtenga un objeto de respuesta.

## TAREA 1:
Solicitar la página Wiki de lanzamiento de Falcon9 desde su URL

Primero, ejecutemos un método HTTP GET para solicitar la página HTML de lanzamiento de Falcon9 como una respuesta HTTP.

In [5]:
# use el método requests.get() con la static_url proporcionada
# asigne la respuesta a un objeto
page=requests.get(static_url)


Crea un objeto BeautifulSoup a partir de la respuesta HTML

In [6]:
# Utilice BeautifulSoup() para crear un objeto BeautifulSoup a partir de un contenido de texto de respuesta
# soup = BeautifulSoup(response, 'html.parser')

soup = BeautifulSoup(page.text, 'html.parser')

Imprima el título de la página para verificar si el objeto BeautifulSoup se creó correctamente

In [7]:
# Utilice el atributo soup.title
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

## TAREA 2:
Extraer todos los nombres de columnas/variables del encabezado de la tabla HTML

A continuación, queremos recopilar todos los nombres de columnas relevantes del encabezado de la tabla HTML

Primero, intentemos encontrar todas las tablas en la página wiki. Si necesita refrescar su memoria sobre BeautifulSoup, consulte el enlace de referencia externa hacia el final de este laboratorio

In [8]:
# Utilice la función find_all en el objeto BeautifulSoup, con tipo de elemento `table`
# Asignar el resultado a una lista llamada `html_tables`
html_tables=soup.find_all('table')


In [9]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 

Debería poder ver los nombres de las columnas incrustados en los elementos del encabezado de la tabla `<th>` de la siguiente manera:

```html
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```

A continuación, solo necesitamos iterar a través de los elementos `<th>` y aplicar la función `extract_column_from_header()` proporcionada para extraer el nombre de la columna uno por uno.

In [10]:
column_names = []
# Aplicar la función find_all() con el elemento `th` en first_launch_table
# Iterar cada elemento th y aplicar la función extract_column_from_header() proporcionada para obtener un nombre de columna
# Agregar el nombre de la columna no vacía (`si name no es None y len(name) > 0`) en una lista llamada column_names
for i in first_launch_table.find_all('th'):
    if extract_column_from_header(i)!=None:
        if len(extract_column_from_header(i))>0:
            column_names.append(extract_column_from_header(i))

Compruebe los nombres de las columnas extraídas

In [11]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TAREA 3:
Crear un marco de datos analizando las tablas HTML de inicio

Crearemos un diccionario vacío con claves de los nombres de columna extraídos en la tarea anterior. Más adelante, este diccionario se convertirá en un marco de datos de Pandas

In [12]:
launch_dict= dict.fromkeys(column_names)

# Eliminar una columna irrelevante
del launch_dict['Date and time ( )']

# Inicialicemos launch_dict con cada valor para que sea una lista vacía
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []

# Agregamos algunas columnas nuevas
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

A continuación, solo tenemos que completar el launch_dict con los registros de lanzamiento extraídos de las filas de la tabla.

Por lo general, las tablas HTML en las páginas Wiki suelen contener anotaciones inesperadas y otros tipos de ruidos, como enlaces de referencia B0004.1[8], valores faltantes N/A [e], formato inconsistente, etc.

Para simplificar el proceso de análisis, proporcionamos un fragmento de código incompleto a continuación para ayudarlo a completar el launch_dict. Complete el siguiente fragmento de código con TODOs o puede optar por escribir su propia lógica para analizar todas las tablas de lanzamiento:

In [13]:
extracted_row = 0
#Extraer cada tabla
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
    # Obtener una fila de la tabla
    for rows in table.find_all("tr"):
        #Verificar si el primer encabezado de la tabla es un número que corresponde al lanzamiento de un número
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #Obtener elemento de tabla
        row=rows.find_all('td')
        #Si es un número, guardar celdas en un diccionario
        if flag:
            extracted_row += 1
            # Valor de número de vuelo
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            # TODO: Añade el número de vuelo a launch_dict con la clave `N.º de vuelo`
            launch_dict['Flight No.'].append(flight_number)
            print(flight_number)
            datatimelist=date_time(row[0])

            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            print(date)

            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            launch_dict['Time'].append(time)
            print(time)

            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict['Version Booster'].append(bv)
            print(bv)

            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            print(launch_site)

            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            print(payload)

            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            print(payload_mass)

            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            print(orbit)

            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            if row[6].a!=None:
                customer = row[6].a.string
            else:
                customer='None'
            launch_dict['Customer'].append(customer)
            print(customer)

            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            print(launch_outcome)

            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)
            print(booster_landing)

            print("******")







1
4 June 2010
18:45
F9 v1.07B0003.18
CCAFS
Dragon Spacecraft Qualification Unit
0
LEO
SpaceX
Success

Failure
******
2
8 December 2010
15:43
F9 v1.07B0004.18
CCAFS
Dragon
0
LEO
NASA
Success
Failure
******
3
22 May 2012
07:44
F9 v1.07B0005.18
CCAFS
Dragon
525 kg
LEO
NASA
Success
No attempt

******
4
8 October 2012
00:35
F9 v1.07B0006.18
CCAFS
SpaceX CRS-1
4,700 kg
LEO
NASA
Success

No attempt
******
5
1 March 2013
15:10
F9 v1.07B0007.18
CCAFS
SpaceX CRS-2
4,877 kg
LEO
NASA
Success

No attempt

******
6
29 September 2013
16:00
F9 v1.17B10038
VAFB
CASSIOPE
500 kg
Polar orbit
MDA
Success
Uncontrolled
******
7
3 December 2013
22:41
F9 v1.1
CCAFS
SES-8
3,170 kg
GTO
SES
Success
No attempt
******
8
6 January 2014
22:06
F9 v1.1
CCAFS
Thaicom 6
3,325 kg
GTO
Thaicom
Success
No attempt
******
9
18 April 2014
19:25
F9 v1.1
Cape Canaveral
SpaceX CRS-3
2,296 kg
LEO
NASA
Success

Controlled
******
10
14 July 2014
15:15
F9 v1.1
Cape Canaveral
Orbcomm-OG2
1,316 kg
LEO
Orbcomm
Success
Controlled
******
1

Después de completar los valores del registro de lanzamiento analizados en launch_dict, puede crear un marco de datos a partir de él.

In [14]:
df=pd.DataFrame(launch_dict)

In [15]:
df.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.07B0003.18,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.07B0004.18,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.07B0005.18,No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,F9 v1.07B0006.18,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,F9 v1.07B0007.18,No attempt\n,1 March 2013,15:10


In [16]:
df.to_csv('spacex_web_scraped.csv', index=False)